<a href="https://colab.research.google.com/github/JimMcDonough/Capstone/blob/main/Roberta_hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov 13 19:35:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 7.5 MB/s 
     |████████████████████████████████| 163 kB 85.4 MB/s 
     |████████████████████████████████| 7.6 MB 77.6 MB/s 


In [4]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 8.2 MB/s 
     |████████████████████████████████| 95 kB 5.7 MB/s 
     |████████████████████████████████| 212 kB 76.3 MB/s 
     |████████████████████████████████| 115 kB 86.1 MB/s 
     |████████████████████████████████| 127 kB 87.2 MB/s 
     |████████████████████████████████| 115 kB 87.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [5]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [6]:
from datasets import load_dataset, Value, Sequence, Features

data_files={"train": r"/content/drive/My Drive/sample_data/train.json",
           "validation": r"/content/drive/My Drive/sample_data/val.json",
           "test": r"/content/drive/My Drive/sample_data/test.json"}

In [7]:
features = Features({'answers': [{'text': Value(dtype='string'), 'answer_start': Value(dtype='int32')}],
'context': Value(dtype='string'),
'id': Value(dtype='string'),
'question': Value(dtype='string'),
'title': Value(dtype='string')})

In [8]:
dataset = load_dataset("json", data_files=data_files, features=features, cache_dir= r"/content/drive/My Drive/sample_data/")

  0%|          | 0/3 [00:00<?, ?it/s]

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [10]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",     #only truncates the context nothing else
        return_offsets_mapping=True,  # map the start and end positions of the answer to the original context 
        padding="max_length",
    )
    
    offset_mapping = inputs.pop("offset_mapping")
    #print(offset_mapping)
    answers = examples["answers"]
    start_positions = []
    end_positions = []
    #print(answers)
    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        #start_char = answer["answer_start"][0]
        #end_char = answer["answer_start"][0] + len(answer["text"][0])  #start char + length giver chars with answer
        start_char = answer[0]["answer_start"]
        end_char = answer[0]["answer_start"] + len(answer[0]["text"])  
        sequence_ids = inputs.sequence_ids(i)  #no clue what this does
        #print(answer, end_char, sequence_ids)
        # Find the start and end of the context        ; not sure how this section works
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
    
        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
            
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)
            
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [11]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [12]:
from transformers import DefaultDataCollator
from transformers import TFAutoModelForQuestionAnswering
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

data_collator = DefaultDataCollator(return_tensors="tf")

In [13]:
batch_sizes = [8,16]
epoch_sizes = [3]
learning_rates = [0.00032, 0.000192, 0.000128, 0.000064]

In [14]:
results_list = []
for num_epochs in epoch_sizes:
    for batch_size in batch_sizes:
        for learning_rate in learning_rates:
            result = {'epochs':num_epochs,
                     'batch_size': batch_size,
                     'learning_rate':learning_rate}
            
            model = TFAutoModelForQuestionAnswering.from_pretrained("roberta-base")
            
            tf_train_set = model.prepare_tf_dataset(
                tokenized_squad["train"],
                shuffle=True,
                batch_size=batch_size,
                collate_fn=data_collator,
            )
            
            tf_validation_set = model.prepare_tf_dataset(
                tokenized_squad["validation"],
                shuffle=False,
                batch_size=batch_size,
                collate_fn=data_collator,
            )
            
            num_train_steps = len(tf_train_set) * num_epochs
            lr_scheduler = PolynomialDecay(
                initial_learning_rate=learning_rate, end_learning_rate=0.0, decay_steps=num_train_steps
            )
            
            opt = Adam(learning_rate=lr_scheduler)
            
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
            model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
            
            history = model.fit(tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)
            result['val_end_accuracy'] = history.history['val_end_logits_accuracy']
            result['val_start_accuracy'] = history.history['val_start_logits_accuracy']
            results_list.append(result) 
            del history
            del model
            del tf_train_set
            del tf_validation_set

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1575/1575 [==============================] - 181s 106ms/step - loss: 11.2588 - end_logits_loss: 5.6185 - start_logits_loss: 5.6403 - end_logits_accuracy: 0.0119 - start_logits_accuracy: 0.0125 - val_loss: 11.9013 - val_end_logits_loss: 5.9507 - val_start_logits_loss: 5.9507 - val_end_logits_accuracy: 0.0057 - val_start_logits_accuracy: 0.0075
Epoch 2/3
1575/1575 [==============================] - 164s 104ms/step - loss: 11.9098 - end_logits_loss: 5.9548 - start_logits_loss: 5.9551 - end_logits_accuracy: 0.0032 - start_logits_accuracy: 0.0025 - val_loss: 11.9013 - val_end_logits_loss: 5.9507 - val_start_logits_loss: 5.9507 - val_end_logits_accuracy: 0.0083 - val_start_logits_accuracy: 0.0077
Epoch 3/3
1575/1575 [==============================] - 164s 104ms/step - loss: 11.9064 - end_logits_loss: 5.9529 - start_logits_loss: 5.9534 - end_logits_accuracy: 0.0026 - start_logits_accuracy: 0.0022 - val_loss: 11.9013 - val_end_logits_loss: 5.9507 - val_start_logits_loss: 5.9507 - val

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
1575/1575 [==============================] - 178s 107ms/step - loss: 11.8671 - end_logits_loss: 5.9334 - start_logits_loss: 5.9337 - end_logits_accuracy: 0.0032 - start_logits_accuracy: 0.0038 - val_loss: 11.9013 - val_end_logits_loss: 5.9507 - val_start_logits_loss: 5.9507 - val_end_logits_accuracy: 0.0073 - val_start_logits_accuracy: 0.0073
Epoch 2/3
1575/1575 [==============================] - 165s 104ms/step - loss: 11.9062 - end_logits_loss: 5.9533 - start_logits_loss: 5.9529 - end_logits_accuracy: 0.0025 - start_logits_accuracy: 0.0023 - val_loss: 11.9013 - val_end_logits_loss: 5.9507 - val_start_logits_loss: 5.9507 - val_end_logits_accuracy: 0.0059 - val_start_logits_accuracy: 0.0073
Epoch 3/3
1575/1575 [==============================] - 165s 105ms/step - loss: 11.9044 - end_logits_loss: 5.9521 - start_logits_loss: 5.9522 - end_logits_accuracy: 0.0025 - start_logits_accuracy: 0.0025 - val_loss: 11.9013 - val_end_logits_loss: 5.9507 - val_start_logits_loss: 5.9507 - val

...

Epoch 1/3
787/787 [==============================] - 158s 188ms/step - loss: 3.5154 - end_logits_loss: 1.7404 - start_logits_loss: 1.7750 - end_logits_accuracy: 0.5481 - start_logits_accuracy: 0.5351 - val_loss: 3.0879 - val_end_logits_loss: 1.3586 - val_start_logits_loss: 1.7293 - val_end_logits_accuracy: 0.6547 - val_start_logits_accuracy: 0.6025
Epoch 2/3
787/787 [==============================] - 144s 184ms/step - loss: 1.9248 - end_logits_loss: 0.9489 - start_logits_loss: 0.9760 - end_logits_accuracy: 0.7338 - start_logits_accuracy: 0.7055 - val_loss: 2.5297 - val_end_logits_loss: 1.1709 - val_start_logits_loss: 1.3587 - val_end_logits_accuracy: 0.6976 - val_start_logits_accuracy: 0.6499
Epoch 3/3
787/787 [==============================] - 144s 184ms/step - loss: 0.9817 - end_logits_loss: 0.4843 - start_logits_loss: 0.4974 - end_logits_accuracy: 0.8523 - start_logits_accuracy: 0.8307 - val_loss: 2.8151 - val_end_logits_loss: 1.3326 - val_start_logits_loss: 1.4824 - val_end_logits_

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
787/787 [==============================] - 158s 188ms/step - loss: 3.1068 - end_logits_loss: 1.5262 - start_logits_loss: 1.5806 - end_logits_accuracy: 0.5979 - start_logits_accuracy: 0.5765 - val_loss: 2.4416 - val_end_logits_loss: 1.1636 - val_start_logits_loss: 1.2780 - val_end_logits_accuracy: 0.6863 - val_start_logits_accuracy: 0.6405
Epoch 2/3
787/787 [==============================] - 145s 184ms/step - loss: 1.6498 - end_logits_loss: 0.8123 - start_logits_loss: 0.8375 - end_logits_accuracy: 0.7669 - start_logits_accuracy: 0.7400 - val_loss: 2.3671 - val_end_logits_loss: 1.1188 - val_start_logits_loss: 1.2483 - val_end_logits_accuracy: 0.6960 - val_start_logits_accuracy: 0.6626
Epoch 3/3
787/787 [==============================] - 145s 184ms/step - loss: 0.8894 - end_logits_loss: 0.4448 - start_logits_loss: 0.4446 - end_logits_accuracy: 0.8640 - start_logits_accuracy: 0.8435 - val_loss: 2.6929 - val_end_logits_loss: 1.2633 - val_start_logits_loss: 1.4296 - val_end_logits_

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
787/787 [==============================] - 158s 188ms/step - loss: 2.9071 - end_logits_loss: 1.4321 - start_logits_loss: 1.4750 - end_logits_accuracy: 0.6227 - start_logits_accuracy: 0.5971 - val_loss: 2.2345 - val_end_logits_loss: 1.0570 - val_start_logits_loss: 1.1775 - val_end_logits_accuracy: 0.7157 - val_start_logits_accuracy: 0.6667
Epoch 2/3
787/787 [==============================] - 145s 184ms/step - loss: 1.4940 - end_logits_loss: 0.7269 - start_logits_loss: 0.7671 - end_logits_accuracy: 0.7879 - start_logits_accuracy: 0.7590 - val_loss: 2.2348 - val_end_logits_loss: 1.0443 - val_start_logits_loss: 1.1905 - val_end_logits_accuracy: 0.7238 - val_start_logits_accuracy: 0.6821
Epoch 3/3
787/787 [==============================] - 145s 184ms/step - loss: 0.8684 - end_logits_loss: 0.4190 - start_logits_loss: 0.4494 - end_logits_accuracy: 0.8671 - start_logits_accuracy: 0.8424 - val_loss: 2.4820 - val_end_logits_loss: 1.1666 - val_start_logits_loss: 1.3153 - val_end_logits_